# MOD 300: Assignment 3

**Basel Abu Sablih, Anders Bygdås and Aleksander Bø**

Date: **Oct 27, 2025**


In [13]:
# Import required packages
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import dna

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TOPIC 1: Calculate DNA volume via Monte Carlo simulation.


**Task 0**

We define a simulation box in 3d space by specifying its upper bound. Since we are dealing with DNA, we chose to use nanometer as unit.


In [14]:
box = dna.SimulationBox(20,20,20)

**Task 1**


We create the random point...


In [15]:
point = dna.create_random_point(box)

**Task 2**

We create the random sphere...


In [ ]:
sphere = dna.create_random_sphere(box)
print(sphere)


SPHERE X: 9.27 Y: 10.03 Z: 8.4 Radius: 7.68
POINT Center: X 3.7 Y 6.72 Z 11.93
Is Point inside Sphere:  True


**Task 3**

We check if the point is inside the sphere


In [ ]:
point = dna.create_random_point(box)
print(point)

print("Is Point inside Sphere: ", sphere.is_point_inside(point))

**Task 4**


In [ ]:
# number of randomly generated points
n_points = 10000

points = []
for i in range(n_points):  
    points.append(dna.create_random_point(box))

n_points_inside = 0
for p in points:
    if sphere.is_point_inside(p):
        n_points_inside += 1

fraction = n_points_inside / n_points
print("Fraction of points inside: ", fraction)

Fraction of points inside:  0.3
